### **Presenting the Results**

Most of the implementations are in the script file, here we just assemble everything together and present the results for view. 
This is necessary since the script is very long. 
Run the cell below to import everthing from the pre-cooked script file. 



In [1]:
from sqp import *

TESTSTRING = "02002102000101000000102002110000021102201100000121101100102200102212002011112010100202020220021011110001020002102020211001102210020111001100000102100022100110201210022100020000101002210202100021000220220211202211110002221011010211000211202201021002102200012201101110222110002022012210202020020102100202211110202001122020000110020222220022110010020002102120002010010000211002021102102121210202221122000110202101020002020022200021000211020211022210200121022200010211002201101110220220110202110202210020212102102120002210002202112110210020001010002002000202102121222022121022201210211202020022100222101102112100221202021001010211020210102110202211200202000000000022102020000021111220012110201121010002002020000120200222022110202011002101002110010002120221100011000002100220222202021110222102200022001101011122021021111120021100010210222100222110202210102002221000021202020210200201101001120002211121011000212002000122022200121011120000210111011111020112221002002202"
# TESTSTRING = TESTSTRING[:40]
numpy.printoptions(suppress=True)

The following code uses what is written in the script file and call the SLSQP solver from scipy and solve the problem for a specific strings. 

In [2]:
def run_sqp(test_string, lmbd):
    global pbm
    pbm = ProblemModelingSQP(test_string, lmbd=lmbd)
    n = len(pbm.States)
    # Functional representation of Constraints =================================
    ineq_cons = {'type': 'ineq',
            'fun' : pbm.IneqCon,
            'jac' : pbm.IneqJac
        }
    eq_cons = {'type': 'eq',
            'fun' : pbm.EqnCon,
            'jac' : pbm.EqnJac
        }
    l = pbm.CRowsCount + len(pbm.TransFreqasVec)
    bounds = scipy.optimize.Bounds(
        np.zeros(l), np.full(l, np.inf)
    )
    # the objective function and gradient ======================================
    objfxn = pbm.ObjFxn
    objgrad = pbm.ObjGrad
    # initial Guess ============================================================
    C = pbm.ConstraintMatrixC
    x0 = pbm.TransitionMatrix.reshape(-1)
    # x0 = np.random.rand(n**2)
    x0 = np.hstack((x0, np.dot(C, x0))).copy()
    global res
    res = scipy.optimize.minimize(
            objfxn, x0, method='SLSQP', jac=objgrad,
            constraints=[eq_cons, ineq_cons], 
            options={'ftol': 1e-12, 'disp': True, "maxiter":5000},
            bounds=bounds, 
        )
    M = res.x[:n**2].reshape((n, n))
    return M, pbm.TransitionMatrix


The above function takes in a test string consist of characters representing the observed states from a markov chain. 
It will print out the optimize state transition matrix. 
It returns the optimize state transition matrix together with the original state transition matrix estimated via MLE. 
It setup the problem using the code written in the imported script, and then it solves using the `scipy.optimize.minimize` module using sequential quadratic programming. 
Each time the funtion is evaluated, it prints out the objective value of the function and a timestep accurated to miliseconds. 
In the cell below, we test it with some basic input. 

In [12]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.0003)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)
print("Eqn con rhs: ")
print(pbm.EqnCon(res.x))
print("Ineq con rhs: ")
print(pbm.IneqCon(res.x))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3359.318709708161
            Iterations: 105
            Function evaluations: 1031
            Gradient evaluations: 103
The original MLE estimated transition matrix is: 


array([[0.27645051, 0.33788396, 0.38566553],
       [0.15444015, 0.34362934, 0.5019305 ],
       [0.42298289, 0.17359413, 0.40342298]])

The New Transiiton Matrix after the SQP is: 


array([[0.97449239, 0.01275381, 0.01275381],
       [0.97449239, 0.01275381, 0.01275381],
       [0.01275381, 0.97449239, 0.01275381]])

Eqn con rhs: 
[-1.11022302e-16  0.00000000e+00  2.22044605e-16]
Ineq con rhs: 
[1.28022593e-14 1.08758488e-14 1.00102218e-14 9.70924730e-15
 9.57111994e-15 9.00798533e-15 9.16107468e-15 9.38008352e-15
 7.58014528e-15 3.63320920e-03 9.22319946e-15 7.82714080e-15
 9.35909743e-15 4.00796675e-03 9.57898040e-15 2.96426133e-03
 8.04467170e-15 7.95217570e-15 1.11877060e-14 3.19706543e-03
 1.03110066e-14 9.91502887e-15 8.71872019e-15 1.01038969e-14
 1.08183184e-14 1.10805462e-14 1.16060488e-14 7.80509224e-15
 3.88685296e-03 1.09084180e-14 9.96495638e-15 2.19188598e-03
 9.13745262e-15 2.77569024e-03 8.54829026e-15 1.14073247e-14
 9.93662527e-03 5.80377791e-03 1.02704303e-14 9.13037570e-03
 3.03004788e-03 4.44128512e-03 9.51062146e-15 5.05666144e-03
 7.69545018e-15 9.99244090e-15 8.83483824e-15 7.67928950e-15
 9.18353799e-15 1.27159568e-14 9.35506555e-15 1.12945677e-14
 7.85932734e-15 7.70023422e-15 1.06567993e-14 8.92059863e-15
 9.21699240e-15 3.53532604e-03 2.08883741e-03 2.60368116e-03
 1.104

We now decrease the value of lamabda, the reguarlizatin term and see how it affects the results of the transition matrix. 

In [9]:
M, StateTransMatrix = run_sqp(TESTSTRING, 0.0334)
res
print("The original MLE estimated transition matrix is: ")
display(StateTransMatrix)
print("The New Transiiton Matrix after the SQP is: ")
display(M)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 1055.766409410068
            Iterations: 47
            Function evaluations: 480
            Gradient evaluations: 47
The original MLE estimated transition matrix is: 


array([[0.27645051, 0.33788396, 0.38566553],
       [0.15444015, 0.34362934, 0.5019305 ],
       [0.42298289, 0.17359413, 0.40342298]])

The New Transiiton Matrix after the SQP is: 


array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])